### Les librairies

In [322]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from datetime import date
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re

import numpy as np

### Mes fonctions

In [369]:
############## Mes fonctions ####################################

### ajouter un delay
def random_delay():
    time.sleep(random.uniform(2, 5))


### enlever les baslises html
def remove_html_tags(text_col):
    if text_col is None:
        return None
    else:
        clean = re.compile('<.*?>')
        return re.sub(clean,"", text_col)
    

### convertir l'année de création en année
def convert_year_to_date(value):
    try:
        # Convertir la valeur en int, puis en string et enfin en datetime avec le format %Y
        year_str = str(int(value))
        return pd.to_datetime(year_str, format='%Y').strftime('%Y')
    except:                                                                       # Je retourne la valeur originale si ce n'est pas une date
      return value


### changer le format de la date de publication %Y-%m-%d
def convert_publication_date(value):
    try:
        # Convertir la valeur en int, puis en string et enfin en datetime avec le format %Y
        return pd.to_datetime(value).strftime('%Y-%m-%d')
    except:                                                                       # Je retourne la valeur originale si ce n'est pas une date
      return value


### remplacer les catégories de niveau d'expérience
def experience_replace(experience):
    lower_exp = experience.lower()
    small_exp = lower_exp.replace("_", " ")
    return(small_exp)

# ### décompacter les tools et les skills
# # note : j'enregistre les tools et les skills dans une même liste que je vais ensuite mettre dans une colonne en appelant la fonction jtools
# # puis je droperai la colonne des skills qui reprendra la même liste en doublon

# skills = []                                            # je crée un liste pour stocker mes outils et mes skills
# def jtools_extractor(tools_list):
                                     
#     if isinstance(tools_list, list):                   # je vérifie que tools_list est une liste
#         for dicos in tools_list:                       # je parcours chaque dictionnaire dans la liste des outils
#             for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
#                 if key == "name":                       # si la clé est = à name
#                     skills.append(dicos["name"])          # je stocke l'outil dans la liste
#             if skills:  # je vérifie si des outils ont été ajoutées à la liste
#                 return skills
#         else:
#             return None  # je retourne None si aucun outil n'a été trouvé


# ### décompacter les skills
# def jskills_extractor(skills_list):
#                                             # je crée une liste pour stocker les compétences
#     if isinstance(skills_list, list):                   # je vérifie que skills_list est une liste
#         for dictos in skills_list:                      # je parcours chaque dictionnaire dans la liste des compétences
#             if 'name' in dictos:                        # je vérifie que la clé 'name' existe dans le dictionnaire
#                 value = dictos['name']
#                 if 'fr' in value:                       # je vérifie que la clé 'fr' existe dans le sous-dictionnaire 'name'
#                     skills.append(value['fr'])          # je stocke la compétence dans la liste
#             if skills:                                  # je vérifie si des compétences ont été ajoutées à la liste
#                 return skills
#         else:
#             return None                                 # je retourne None si aucune compétence en français n'a été trouvée
#     else:
#         return None                                     # je retourne None si skills_list n'est pas une liste

def extract_competences(row):
    skills_col = 'job.skills'                            # je définis les colonnes sur lesquelles je vais passer ma fonction
    tools_col = 'job.tools'                              # directement dans la fonction pour pouvoir la passer sur les 2
    competences = []                                     # j'initialise la liste où je vais enregistrer le tout
    
    # Extraire les outils
    tools_list = row[tools_col]
    if isinstance(tools_list, list):                           # je vérifie que tools_list est une liste
        for dicos in tools_list:                               # je parcours chaque dictionnaire dans la liste des outils
            for key, tool in dicos.items():                    # dans le duo key/value de mes items de chaque dico
                if key == "name":                              # si la clé est = à name
                    competences.append(dicos["name"])          # je stocke l'outil dans la liste
    
    # Extraire les compétences
    skills_list = row[skills_col]
    if isinstance(skills_list, list):                           # je vérifie que skills_list est une liste
        for dictos in skills_list:                              # je parcours chaque dictionnaire dans la liste des compétences
            if 'name' in dictos:                        # je vérifie que la clé 'name' existe dans le dictionnaire
                value = dictos['name']
                if 'fr' in value:                       # je vérifie que la clé 'fr' existe dans le sous-dictionnaire 'name'
                    competences.append(value['fr'])          # je stocke la compétence dans la liste
    
    return competences if competences else None         # je vérifie si des compétences ont été ajoutées à la liste
    
### décompacter les urls des annonces
def jurls_extractor(urls_list):
    if isinstance(urls_list, list):                        # je vérifie que urls_list est une liste
        for dics in urls_list:                             # je parcours chaque dictionnaire dans la liste
            if isinstance(dics, dict) and 'href' in dics:  # je vérifie que dics est un dictionnaire et contient 'href'
                return dics['href']                        # je retourne l'URL
    return None                                            # Retourne None si aucun URL n'a été trouvée

### décompacter les secteurs
def sector_extractor(sectors_list):
    sectors = []                                      # je crée un liste pour stocker mes outils
    if isinstance(sectors_list, list):                   # je vérifie que tools_list est une liste
        for dictos in sectors_list:                       # je parcours chaque dictionnaire dans la liste des outils
            for key, sector in dictos.items():             # dans le duo key/value de mes items de chaque dico
                if key == "name" or key == "parent_name":                       # si la clé est = à name
                    sectors.append(dictos["name", "parent_name"])          # je stocke l'outil dans la liste
            if sectors:  # je vérifie si des outils ont été ajoutées à la liste
                return sectors
        else:
            return None  # je retourne None si aucun outil n'a été trouvé

### récupérer ce qui est obligatoire pour la candidature
# def appli_mandatory_extractor(mandatory_list):
#     applications = []                                           # je crée un liste pour stocker mes éléments mandatory
#     if isinstance(mandatory_list, list):                        # je vérifie que skills_list est une liste
#         for dicts in mandatory_list:                            # je vais chercher chaque dico contenu dans cette liste
#             for key, value in dicts.items():                    # dans le duo key/value de mes items de chaque dico
#                 if value == "mandatory":                        # si une value est = à mandatory
#                     applications.append(dicts["name"])          # alors je stocke chaque clé correspondante à name
#             if applications:  # je vérifie si des outils ont été ajoutées à la liste
#                 return applications
#         else:
#             return None  # je retourne None si aucun outil n'a été trouvé
#     return(applications) 

### Remplacer les noms anglais de CDI/CDD
types_contrats = {'full_time': 'CDI', 'temporary': 'CDD'}

# get est une méthode de dict. elle va chercher la clé passée en argument de la fonction
# si pas égal alors retourne la clé elle-même (d'où le double contrat)
def contract_corrector(contrat):
    return types_contrats.get(contrat, contrat)                 


### Récupérer les départements
def dpt_extractor(zipcode):
    departement = re.search(r"\A\d{2}", zipcode)                # 2 {2}    premiers \A    chiffres \d - le r permet de ne pas prendre \ pour un échappement
    return departement.group(0) if departement else None

### Procèdure

1. Aller sur la page de résultats avec selenium
2. Récupérer les URL pour aller de page en page
3. Faire une fonction pour aller récupérer les URL de chaque annonce sur chaque page
4. Couper les URL en relatif puis les reconcatener pour les passer dans l'API
5. Itérer pour requêter l'API avec chaque URL récupérée sur chaque page de résultats
6. Transformer en dataframe, décompacter et dropper ce qui doit l'être
7. Exporter en CSV avant l'EDA

### 1. Récupération des URL des pages et des jobs

Je commence par lancer le code pour récupérer les pages de résultats

In [38]:
############## SCRIPT PRINCIPAL COMPLET 1 ####################################
##############################################################################

## 1. je lance selenium pour aller sur la page de résultats ########################################################################
service = Service(executable_path = 'chromedriver.exe')                         # je dis où est mon driver                       
driver = webdriver.Chrome(service=service)                                      # je définis mon driver
driver.get("https://www.welcometothejungle.com/fr")                                                         # et je l'envoie sur la page d'accueil

wait = WebDriverWait(driver, 8)                                                 # je laisse le temps à la page de se charger

input_element = driver.find_element(By.ID,"search-query-field")                 # je sélectionne la barre de recherche
input_element.clear()                                                           # je la vide au cas où elle est pré-remplie                                             
input_element.send_keys("Data analyst" + Keys.ENTER)                            # j'écris dans la barre de recherche

random_delay()                                                                     # je laisse le temps à la page de charger

## 2. je vais chercher les url de pagination ######################################################################################
page_source = driver.page_source                                   # je vais chercher la page avec mon driver
soup = BeautifulSoup(page_source, "html.parser")                   #  je passe le code HTML à BeautifulSoup pour l'analyser

soup_pages = soup.find('nav', {"aria-label" : "Pagination"})      # Puis je vais chercher la section nav :

if soup_pages is not None:                                        # Je mets un if si la page n'est pas chargée et que nav est vide
    pli = soup_pages.find_all('li')                               # Puis je vais chercher tous les li contenus dans cette nav
    nb_pages = pli[-2].text                                       # Puis je vais chercher le texte de l'avant dernier li 
    print(f"\nJ'ai {nb_pages} pages")                               # (car le dernier c'est la flèche) pour avoir le nombre de pages

else:                                                             # si la page n'a pas eu le temps de charger, j'affiche un message
    print("Aucun élément HTML correspondant au sélecteur spécifié n'a été trouvé.")

result_url = driver.current_url                                   # je récupère l'URL sur laquelle je suis
base_url = result_url.replace("1","")

base_url = "https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page="
page_number = 1
urls_to_scrape = []

# 3. je fais une boucle qui itère le nom de la page autant de fois que nécessaire
for i in range(1,int(nb_pages)):
    url = f"{base_url}{i}"
    urls_to_scrape.append(url)
print(urls_to_scrape)

random_delay()     

jobs_list = []

for each_page in urls_to_scrape:
    try:
    # je charge la page actuelle dans le navigateur Web
      driver.get(each_page)
      random_delay() 

     # j'attends que la page soit entièrement chargée
      wait = WebDriverWait(driver, 20)
      wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, "Data")))

      links = (driver.find_elements(By.PARTIAL_LINK_TEXT, "Data")        # avec Selenium, je cherche les liens avec data dedans quelque soit la casse 
        or driver.find_elements(By.PARTIAL_LINK_TEXT, "DATA")
        or  driver.find_elements(By.PARTIAL_LINK_TEXT, "data"))

      urls = [link.get_attribute("href") for link in links]              # j'extraits le contenu de href pour le mettre dans la liste urls
      print(urls)
    
      jobs_list.extend(urls)
    except:
        print(f"An error occured on page {driver.current_url}")  

print(f"J'ai {len(jobs_list)} annonces au total")
print(jobs_list)

driver.quit()                                                     # je ferme le navigateur une fois tout fini



J'ai 34 pages
['https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=1', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=2', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=3', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=4', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffice

### 2. Récupération des infos de toutes les annonces

2.1 Formatage des URL pour l'API

In [39]:
## 1. formatage des urls en relative pour le requêtage ##

relative_urls = [sub.replace("https://www.welcometothejungle.com/fr/companies", "") for sub in jobs_list]
print(relative_urls)
print(len(relative_urls))                               # je vérifie que je n'ai rien perdu

## 2. formatage des urls relatives en url de requêtage ##
api_urls = []
for url in relative_urls:
    api_url = f"https://api.welcometothejungle.com/api/v1/organizations{url}"
    api_urls.append(api_url)
print(api_urls)
print(len(api_urls))                       # je vérifie que je n'ai rien perdu

['/esri/jobs/esri-france-data-scientist-specialise-analyse-spatiale-et-geomarketing-f-h_meudon?q=4143f6f511597ce85a131a249d7dce4f&o=60953789-64cb-4951-a2f6-ba1a20858ed1', '/sport-heroes-group/jobs/senior-data-analyst-head-of-data_paris?q=4143f6f511597ce85a131a249d7dce4f&o=dcb9d6c5-bb31-46aa-8c35-6e9f0168b3e2', '/carrefour/jobs/data-analyst-f-h_evry_CARRE_NPVOZeo?q=4143f6f511597ce85a131a249d7dce4f&o=ee9b39cd-deef-40d5-988b-57748c7c8132', '/hublo/jobs/senior-data-analyst-f-h-n_paris_HUBLO_bR85090?q=4143f6f511597ce85a131a249d7dce4f&o=0b5d40e8-64f9-4643-969b-752d0fd2582c', '/toucan-toco/jobs/data-analyst-engineer-revenue-operations_paris?q=4143f6f511597ce85a131a249d7dce4f&o=acf940dd-2e9c-4ef2-935b-30c2caf6b34f', '/rent-a-car/jobs/data-analyst-rh-joinville-le-pont-h-f_joinville-le-pont?q=4143f6f511597ce85a131a249d7dce4f&o=d6715289-a3b1-41c0-b3cf-48b13b14abaa', '/fbd-group/jobs/data-analyst-f-h_tremblay-en-france?q=4143f6f511597ce85a131a249d7dce4f&o=270b55d3-fa49-4015-a04e-7ee30b35fe7d', '/k

2.2. Requêtage et fabrication du Dataframe

In [40]:
## 3.  je requête chaque URL et je stocke dans un dataframe ##

# création d'une liste vide pour stocker les données JSON
data_json = []

for job_url in api_urls:
    link = job_url
    r = requests.get(link)
    data_json.append(json.loads(r.text))                # comme j'itère sur chaque url je dois stocker ce résultat dans une liste
    # print(json.dumps(data_json, indent=2))

# j'en fais un dataframe
df_jraw = pd.json_normalize(data_json)
df_jraw

,job.salary_period,job.contract_type,job.application_fields,job.remote,job.is_default,job.apply_url,job.urls,job.updated_at,job.organization.nb_employees,job.organization.equality_indexes.equality_among_highest_earners,...,job.profession.sub_category_name.es,job.profession.sub_category_name.fr,job.profession.sub_category_name.sk,job.profession.sub_category_reference,job.contract_duration_max,job.organization.website_organization.i18n_descriptions.en,job.organization.equality_indexes,job.featured_page,job.organization.gdpr_setting,job.organization.website_organization.i18n_descriptions.es
0,None,full_time,[{'id': '3605cdfd-d0ce-4063-8041-f0e09eb9cb10'...,partial,False,None,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-26T12:26:07Z,242.0,0.0,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,NaN,NaN,NaN,NaN,NaN
1,None,full_time,[{'id': '0a5dbdd8-47a2-479a-a9dc-5988af8929b3'...,partial,False,None,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-26T10:16:29Z,70.0,10.0,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,,NaN,NaN,NaN,NaN
2,monthly,full_time,[{'id': '6eaef475-fea7-4aa5-9a1d-8918d6df0684'...,no,False,https://www.aplitrak.com/?adid=Q2F0aGVyaW5lLk1...,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-26T09:20:49Z,100000.0,NaN,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,NaN,NaN,NaN,NaN,NaN
3,yearly,full_time,[{'id': '72b937f4-0688-4f71-ac41-b62c42ea862b'...,punctual,False,https://careers.hublo.com/jobs/3616361-senior-...,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-26T06:28:48Z,200.0,0.0,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,"<p>At Hublo, <strong>we believe in the power o...",NaN,NaN,NaN,NaN
4,None,full_time,[{'id': 'af6b4e82-b14a-4e5c-9edd-67497867cc20'...,fulltime,False,None,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-26T07:00:08Z,85.0,NaN,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,<p>Toucan is a customer-facing analytics platf...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,None,full_time,[{'id': '5c36f22d-bfcb-44cd-b988-f95554e5b82f'...,punctual,False,https://jobs.keyrus.fr/jobs/1075663-consultant...,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-14T07:53:57Z,3500.0,0.0,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,NaN,NaN,NaN,NaN,NaN
396,yearly,full_time,[{'id': '5113e7ed-4bf5-40e9-9a20-d361706ab0d3'...,partial,False,http://app.mytalentplug.com/redirection.aspx?o...,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-07T18:44:01Z,137000.0,5.0,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,NaN,NaN,NaN,NaN,NaN
397,None,full_time,[{'id': '5413359b-a100-4405-b0a7-714fb6fc8bf5'...,punctual,False,https://jobs.keyrus.fr/jobs/1094548-consultant...,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-14T07:47:43Z,3500.0,0.0,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,NaN,NaN,NaN,NaN,NaN
398,None,full_time,[{'id': 'e7e8bb66-2650-4679-ac20-ee1781cd0bcf'...,partial,False,https://cgi.njoyn.com/corp/xweb/xweb.asp?clid=...,"[{'kind': 'canonical', 'href': 'https://www.we...",2024-06-26T00:22:28Z,14000.0,0.0,...,Datos/Inteligencia empresarial,Données/Business Intelligence,Údaje/obchodné spravodajstvo,data-business-intelligence-yZjY1,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
df_jraw.columns

Index(['job.salary_period', 'job.contract_type', 'job.application_fields',
       'job.remote', 'job.is_default', 'job.apply_url', 'job.urls',
       'job.updated_at', 'job.organization.nb_employees',
       'job.organization.equality_indexes.equality_among_highest_earners',
       ...
       'job.profession.sub_category_name.es',
       'job.profession.sub_category_name.fr',
       'job.profession.sub_category_name.sk',
       'job.profession.sub_category_reference', 'job.contract_duration_max',
       'job.organization.website_organization.i18n_descriptions.en',
       'job.organization.equality_indexes', 'job.featured_page',
       'job.organization.gdpr_setting',
       'job.organization.website_organization.i18n_descriptions.es'],
      dtype='object', length=136)

In [345]:
## 4. je nettoie et je formate ## 

# 4.1 je drop toutes les colonnes dont je n'ai pas besoin
df_jobs_temp = df_jraw.drop(columns=["job.slug", 
                                   "job.videos", 
                                   "job.updated_at",
                                   "job.offices",
                                   "job.apply_url", 
                                   "job.is_default",
                                   'job.organization.has_external_ats',
                                   'job.organization.labels',
                                   'job.organization.logo.url', 
                                   'job.organization.logo.thumb.url',
                                   'job.organization.sectors',
                                   'job.organization.cover_image.small.url',
                                   'job.organization.cover_image.url',
                                   'job.organization.cover_image.medium.url',
                                   'job.organization.cover_image.large.url',
                                   'job.organization.cover_image.social.url',
                                   'job.organization.website_organization.slug',
                                   'job.organization.website_organization.i18n_descriptions.fr',
                                   'job.organization.gdpr_setting.application_message',
                                   'job.organization.gdpr_setting.consent_duration',
                                   'job.organization.gdpr_setting.privacy_policy_url',
                                   'job.organization.media_website_url',
                                   'job.organization.headquarter.address',
                                   'job.organization.headquarter.local_address',
                                   'job.organization.headquarter.city',
                                   'job.organization.headquarter.latitude',
                                   'job.organization.headquarter.longitude',
                                   'job.organization.headquarter.district',
                                   'job.organization.headquarter.country_code',
                                   'job.organization.headquarter.zip_code',
                                   'job.organization.headquarter.local_city',
                                   'job.organization.headquarter.local_district',
                                   'job.organization.media_facebook',
                                   'job.organization.media_instagram',
                                   'job.organization.media_linkedin',
                                   'job.organization.media_pinterest',
                                   'job.organization.media_twitter',
                                   'job.organization.media_youtube',
                                   'job.organization.playlist_id',
                                   'job.organization.video_playlist_provider',
                                   'job.organization.automatic_email',
                                   'job.archived_at',
                                   'job.profession.name.cs', 'job.profession.name.en',
                                   'job.profession.name.es', 'job.profession.name.sk',
                                   'job.profession.category.cs',
                                   'job.profession.category.en', 'job.profession.category.es',
                                   'job.profession.category.fr', 'job.profession.category.sk',
                                   'job.profession.category_name.cs',
                                   'job.profession.category_name.en',
                                   'job.profession.category_name.es',
                                   'job.profession.category_name.sk',
                                   'job.profession.sub_category_name.cs',
                                   'job.profession.sub_category_name.en',
                                   'job.profession.sub_category_name.es',
                                   'job.profession.sub_category_name.sk',
                                   'job.profession.sub_category_reference',
                                   'job.questions',
                                   "job.featured_page.type",
                                   'job.featured_page.slug',
                                   'job.cta_content.links',
                                   'job.cta_content.contents',
                                   'job.organization.website_organization.i18n_descriptions.en',
                                   'job.organization.website_organization.i18n_descriptions.es',
                                   'job.status',
                                   'job.profession.name.fr',
                                   'job.organization.reference',
                                   'job.organization.default_language',
                                   'job.organization.parity_men',
                                   'job.organization.revenue',
                                   'job.organization.average_age',
                                   'job.salary_currency',
                                   'job.salary_period',
                                   'job.profession.category_reference',
                                   'job.office.address',
                                   'job.office.local_address',
                                   'job.office.district',
                                   'job.office.country_code',
                                   'job.office.local_city',
                                   'job.office.local_district',
                                   'job.language',
                                   'job.start_date',
                                   'job.recruitment_process',
                                   'job.featured_page',
                                   'job.organization.gdpr_setting',
                                   'job.organization.gdpr_setting',
                                   'job.organization.equality_indexes',
                                   'job.organization.equality_indexes.equality_among_highest_earners',
                                   'job.organization.equality_indexes.gap_in_annual_raises',
                                   'job.organization.equality_indexes.gap_in_annual_raises_excluding_promotions',
                                   'job.organization.equality_indexes.gap_in_promotions',
                                   'job.organization.equality_indexes.gender_pay_gap',
                                   'job.organization.equality_indexes.maternity_leave_return_raise',
                                   'job.organization.equality_indexes.published',
                                   'job.organization.equality_indexes.workforce_range',
                                   'job.organization.equality_indexes.year',
                                   'job.organization.turnover',
                                   'job.organization.jobs_count',
                                   'job.organization.website_organization.i18n_descriptions.en',
                                   'job.benefits.FR.categories',
                                   'job.benefits.FR.count',
                                   'job.benefits.FR.preview',
                                   'job.organization.website_organization.i18n_descriptions.es',
                                   'job.apply_url',
                                   "job.organization.profile_type",
                                    'job.contract_duration_max',
                                    'job.contract_duration_min',
                                    'job.profession.category_name.fr',
                                    'job.profession.sub_category_name.fr',
                                    'job.wttj_reference',
                                    'job.company_description',
                                   ]
                                   )



### Les fonctions

In [348]:
# 4.2 je passe toutes mes fonctions

# html cleaner
### faire un if none pass sinon
df_jobs_temp[["job.organization.description_clean","job.profile_clean", "job.description_clean"]] = df_jobs_temp[["job.organization.description","job.profile", "job.description"]].map(remove_html_tags).copy()

# level bins cleaner
df_jobs_temp["job.experience_level_clean"] = df_jobs_temp["job.experience_level"].astype(str).apply(lambda x: experience_replace(x))
df_jobs_temp["job.education_level_clean"] = df_jobs_temp["job.education_level"].astype(str).apply(lambda x: experience_replace(x))

# date convertor
df_jobs_temp['job.organization.creation_year_date'] = df_jobs_temp['job.organization.creation_year'].apply(convert_year_to_date)

# # tools extractor
# df_jobs_temp["competences"] = df_jobs_temp["job.tools"].apply(jtools_extractor)

# # skills extractor
# df_jobs_temp["job.skills_clean"] = df_jobs_temp["job.skills"].apply(jskills_extractor)

df_jobs_temp['competences'] = df_jobs_temp.apply(extract_competences, axis=1)

# job urls extractor
df_jobs_temp["job.urls_clean"] = df_jobs_temp["job.urls"].apply(jurls_extractor)

# # application mandatories
# df_jobs_temp["job.application_mandatory"] = df_jobs_temp["job.application_fields"].apply(appli_mandatory_extractor)

# type de contrats
df_jobs_temp['job.contract_type'] = df_jobs_temp['job.contract_type'].apply(contract_corrector)

# départements
df_jobs_temp["departement"] = df_jobs_temp['job.office.zip_code'].astype(str).apply(dpt_extractor)

# date de publication
df_jobs_temp['date_publication'] = df_jobs_temp['job.published_at'].apply(convert_publication_date)

#### Je calcule le salaire moyen

In [349]:
df_jobs_temp['salaire_avg'] = ((df_jobs_temp['job.salary_max'] + df_jobs_temp['job.salary_min']) / 2)

#### Je concatane dans description_job mes colonne job description et job profile


In [350]:
df_jobs_temp["description_job"] = df_jobs_temp["job.description_clean"].astype(str) + " --- " + df_jobs_temp["job.profile_clean"]

In [178]:
# display(df_jobs_temp["job.urls"][0])

### Je continue de nettoyer

In [352]:
# 4.3 puis je drop les colonnes que j'ai nettoyées
df_jobs = df_jobs_temp.drop(columns=["job.skills", "job.tools", "job.application_fields", "job.profile", "job.organization.description", 
                                     "job.organization.creation_year", "job.urls",'job.profile_clean', 'job.description_clean', 
                                   'job.description',"job.organization.description",'job.published_at','job.experience_level','job.education_level', 'job.office.zip_code',])

In [353]:
df_jobs.shape

(400, 24)

In [354]:
df_jobs.sample(3)

,job.contract_type,job.remote,job.organization.nb_employees,job.organization.equality_indexes.equality_index,job.organization.industry,job.organization.parity_women,job.organization.name,job.reference,job.salary_min,job.name,...,job.organization.description_clean,job.experience_level_clean,job.education_level_clean,job.organization.creation_year_date,competences,job.urls_clean,departement,date_publication,salaire_avg,description_job
362,CDI,partial,4.0,NaN,"IT / Digital, Big Data",NaN,Corail Analytics,CA_5AaNPzX,NaN,Data Analytics Engineer,...,Corail Analytics conseille les entreprises dan...,2 to 3 years,bac 5,2021,"[Aircall, Github, Looker, Tableau, Dbt, Airbyt...",https://www.welcometothejungle.com/fr/companie...,75,2024-06-18,NaN,"Corail Analytics, founded in February 2021, is..."
343,CDI,no,270.0,88.0,"Organization / Management, Strategy, Change Ma...",55.0,Bartle,BARTL_DbRowzm,NaN,Consultant Senior Data Scientist H/F,...,Cabinet de conseil en management interdiscipli...,3 to 4 years,bac 5,2005,None,https://www.welcometothejungle.com/fr/companie...,75,2024-06-03,NaN,"Au sein de l’équipe Bartle Analytics, vous int..."
360,CDI,no,1200.0,NaN,IT / Digital,NaN,Micropole,MICRO_AKLWM6Q,NaN,Consultant Data Gouvernance - F/H,...,Micropole est accélérateur de la transformatio...,2 to 3 years,bac 5,1987,None,https://www.welcometothejungle.com/fr/companie...,69,2024-06-22,NaN,En résumé :Poste : Consultant Data Gouvernance...


In [365]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 24 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   job.contract_type                                 400 non-null    object 
 1   job.remote                                        400 non-null    object 
 2   job.organization.nb_employees                     394 non-null    float64
 3   job.organization.equality_indexes.equality_index  256 non-null    float64
 4   job.organization.industry                         400 non-null    object 
 5   job.organization.parity_women                     262 non-null    float64
 6   job.organization.name                             400 non-null    object 
 7   job.reference                                     400 non-null    object 
 8   job.salary_min                                    49 non-null     float64
 9   job.name             

#### Je renomme mes colonnes

In [361]:
df_jobs2 = df_jobs.rename(columns={"job.contract_type": "contrat", 
                                   "job.remote": "teletravail", 
                                   "job.organization.nb_employees": "nb_employes", 
                                   "job.organization.equality_indexes.equality_index": "index_egalite",
                                   'job.organization.industry': "secteur", 
                                   'job.organization.parity_women': "repartition_genre",
                                   'job.organization.name': "nom_entreprise",
                                   'job.reference': "reference",
                                   'job.salary_min': "salaire_min", 
                                   'job.name': "nom_emploi", 
                                   'job.organization.description_clean': "description_cie",
                                   'job.salary_max': "salaire_max", 
                                   'job.office.city': "ville", 
                                   'job.office.latitude': "office_latitude",
                                   'job.office.longitude': "office_longitude", 
                                   'job.experience_level_clean': "niveau_experience",
                                   'job.education_level_clean': "niveau_etudes", 
                                   'job.organization.creation_year_date': "company_creation",
                                   'job.urls_clean': "url",
                                   })

In [362]:
df_jobs2["impact"] = np.nan

In [366]:
df_jobs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   contrat            400 non-null    object 
 1   teletravail        400 non-null    object 
 2   nb_employes        394 non-null    float64
 3   index_egalite      256 non-null    float64
 4   secteur            400 non-null    object 
 5   repartition_genre  262 non-null    float64
 6   nom_entreprise     400 non-null    object 
 7   reference          400 non-null    object 
 8   salaire_min        49 non-null     float64
 9   nom_emploi         400 non-null    object 
 10  salaire_max        48 non-null     float64
 11  ville              400 non-null    object 
 12  office_latitude    382 non-null    float64
 13  office_longitude   382 non-null    float64
 14  description_cie    400 non-null    object 
 15  niveau_experience  400 non-null    object 
 16  niveau_etudes      400 non

#### Je réordonne les colonnes

In [364]:
df_wtjj = df_jobs2[['nom_entreprise',
                   'nom_emploi',
                    'description_cie',
                    'description_job',
                    'salaire_min',
                    'salaire_max',
                    'salaire_avg',
                    'ville',
                    'departement',
                    'contrat',
                    'niveau_experience',
                    'niveau_etudes',
                    'teletravail',
                    'competences',
                    'date_publication',
                    'secteur',
                    'nb_employes',
                    'url',
                    'impact',
                    'index_egalite',
                    'repartition_genre',
                    'company_creation',
                    'reference',
                    'office_latitude',
                    'office_longitude']]



In [367]:
df_wtjj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nom_entreprise     400 non-null    object 
 1   nom_emploi         400 non-null    object 
 2   description_cie    400 non-null    object 
 3   description_job    279 non-null    object 
 4   salaire_min        49 non-null     float64
 5   salaire_max        48 non-null     float64
 6   salaire_avg        48 non-null     float64
 7   ville              400 non-null    object 
 8   departement        397 non-null    object 
 9   contrat            400 non-null    object 
 10  niveau_experience  400 non-null    object 
 11  niveau_etudes      400 non-null    object 
 12  teletravail        400 non-null    object 
 13  competences        275 non-null    object 
 14  date_publication   400 non-null    object 
 15  secteur            400 non-null    object 
 16  nb_employes        394 non

#### Je fais les exports csv

In [373]:
# 5. j'exporte en csv avec date et heure du moment

# today = date.today()
now = datetime.now()

# Étape 2: je formate la date dd-mm-hh-mn
formatted_date = now.strftime("%d-%m_%H-%M")

In [374]:
# 5.1. j'exporte le df complet pour la viz

# Étape 3: j'ajoute la date au nom de fichier
full_file = f"fullwttj_{formatted_date}.csv"

df_wtjj.to_csv(full_file, index=False)

In [376]:
# 5.2 j'exporte le df strict pour le site
# pour cela je drop qq colonnes

df_strict_wtjj = df_wtjj.drop(columns=['nom_entreprise',
                   'nom_emploi',
                    'description_cie',
                    'description_job',
                    'salaire_min',
                    'salaire_max',
                    'salaire_avg',
                    'ville',
                    'departement',
                    'contrat',
                    'niveau_experience',
                    'niveau_etudes',
                    'teletravail',
                    'competences',
                    'date_publication',
                    'secteur',
                    'nb_employes',
                    'url',
                    'impact',
                    'index_egalite',
                    'repartition_genre'])

# Étape 3: j'ajoute la date au nom de fichier
strict_file = f"strictwttj_{formatted_date}.csv"

df_wtjj.to_csv(strict_file, index=False)